In [ ]:
pip install mysql-connector-python

Note: you may need to restart the kernel to use updated packages.


In [ ]:
pip install matplotlib

Note: you may need to restart the kernel to use updated packages.


In [ ]:
pip install seaborn

Note: you may need to restart the kernel to use updated packages.


In [ ]:
pip install sklearn

Note: you may need to restart the kernel to use updated packages.


In [ ]:
pip install fancyimpute

Note: you may need to restart the kernel to use updated packages.


## PREPARING CO2 DATA FOR INTEGRATION

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline
matplotlib.style.use('ggplot')

In [ ]:
from fancyimpute import KNN

In [ ]:
df = pd.read_csv('owid-co2-data.csv')
weather = pd.read_csv('GlobalLandTemperaturesByCountry.csv')
crops = pd.read_csv('psd_grains_pulses.csv')

In [ ]:
dfe = df.drop(df.iloc[:, 23:46], axis = 1)

In [ ]:
dfe2 = dfe.drop(dfe.columns[[3,4,7,9,11,13,15,17,18,19,20,21,23,25,27,29,31,35,36]], axis = 1)

In [ ]:
dfe2.isnull().sum()

iso_code                       3231
country                           0
year                              0
trade_co2                     21215
cement_co2                    12943
coal_co2                       8003
flaring_co2                   20809
gas_co2                       16346
oil_co2                        4652
other_industry_co2            23192
consumption_co2               21215
total_ghg                     19540
total_ghg_excluding_lucf      19540
methane                       19536
nitrous_oxide                 19536
population                     2299
gdp                           11653
primary_energy_consumption    16501
dtype: int64

In [ ]:
dfe2.shape

(25191, 18)

In [ ]:
(dfe2 == 0).sum()

iso_code                        0
country                         0
year                            0
trade_co2                     150
cement_co2                     21
coal_co2                      102
flaring_co2                   120
gas_co2                       104
oil_co2                        58
other_industry_co2             17
consumption_co2                 0
total_ghg                       3
total_ghg_excluding_lucf        0
methane                        64
nitrous_oxide                 241
population                      0
gdp                             0
primary_energy_consumption      6
dtype: int64

In [ ]:
# Feature Analysis for CO2 Data

# IGNORE ISO CODE, COUNTRY, AND YEAR
# GDP CANNOT BE NEGATIVE
# POPULATION CANNOT BE ZERO
# ALL EMISSION ATTRIBUTES CAN BE 0 OR NEGATIVE

In [ ]:
yearIndex = dfe2[dfe2['year'] < 1960].index
yearIndex2 = dfe2[dfe2['year'] > 2012].index

In [ ]:
dfe3 = dfe2.drop(yearIndex)
dfe4 = dfe3.drop(yearIndex2)

In [ ]:
isoIndex = dfe4[dfe4['iso_code'].isnull()].index
dfe5 = dfe4.drop(isoIndex)

In [ ]:
w_country = weather['Country'].unique()
co2_country = dfe5['country'].unique()
crop_country = crops['Country_Name'].unique()
country_list = [w_country, co2_country, crop_country]

In [ ]:
from functools import reduce
values = reduce(np.intersect1d, country_list)

In [ ]:
values

array(['Afghanistan', 'Albania', 'Algeria', 'Angola', 'Argentina',
       'Armenia', 'Australia', 'Azerbaijan', 'Bahrain', 'Bangladesh',
       'Barbados', 'Belarus', 'Benin', 'Bhutan', 'Bolivia', 'Botswana',
       'Brazil', 'Bulgaria', 'Burkina Faso', 'Burundi', 'Cambodia',
       'Cameroon', 'Canada', 'Central African Republic', 'Chad', 'Chile',
       'China', 'Colombia', 'Costa Rica', 'Croatia', 'Cuba', 'Cyprus',
       'Djibouti', 'Dominican Republic', 'Ecuador', 'Egypt',
       'El Salvador', 'Eritrea', 'Estonia', 'Ethiopia', 'Fiji', 'Gabon',
       'Georgia', 'Ghana', 'Guatemala', 'Guinea', 'Guyana', 'Haiti',
       'Honduras', 'Hong Kong', 'Hungary', 'Iceland', 'India',
       'Indonesia', 'Iran', 'Iraq', 'Israel', 'Jamaica', 'Japan',
       'Jordan', 'Kazakhstan', 'Kenya', 'Kuwait', 'Kyrgyzstan', 'Laos',
       'Latvia', 'Lebanon', 'Lesotho', 'Liberia', 'Libya', 'Lithuania',
       'Madagascar', 'Malawi', 'Malaysia', 'Mali', 'Malta', 'Mauritania',
       'Mauritius', 'Mexico'

In [ ]:
dfe6 = dfe5[dfe5.country.isin(values) == True]

## IMPUTING MISSING VALUES IN THE CO2 DATASET WITH KNN

In [ ]:
(dfe6 == 0).sum()

iso_code                       0
country                        0
year                           0
trade_co2                     65
cement_co2                     0
coal_co2                      17
flaring_co2                   46
gas_co2                       48
oil_co2                        0
other_industry_co2            13
consumption_co2                0
total_ghg                      0
total_ghg_excluding_lucf       0
methane                        0
nitrous_oxide                  0
population                     0
gdp                            0
primary_energy_consumption     0
dtype: int64

In [ ]:
dfe6.isnull().sum()

iso_code                         0
country                          0
year                             0
trade_co2                     4838
cement_co2                    1339
coal_co2                      2288
flaring_co2                   4762
gas_co2                       3197
oil_co2                         55
other_industry_co2            6395
consumption_co2               4838
total_ghg                     4041
total_ghg_excluding_lucf      4041
methane                       4040
nitrous_oxide                 4040
population                       0
gdp                            654
primary_energy_consumption    2076
dtype: int64

In [ ]:
numeric_cols = dfe6.select_dtypes(include='number').columns.tolist()

In [ ]:
dfe6filled = dfe6.copy()

In [ ]:
dfe6filled[numeric_cols] = KNN(k=10).fit_transform(dfe6[numeric_cols])

Imputing row 1/7122 with 10 missing, elapsed time: 6.282
Imputing row 101/7122 with 2 missing, elapsed time: 6.306
Imputing row 201/7122 with 4 missing, elapsed time: 6.328
Imputing row 301/7122 with 2 missing, elapsed time: 6.350
Imputing row 401/7122 with 7 missing, elapsed time: 6.370
Imputing row 501/7122 with 8 missing, elapsed time: 6.394
Imputing row 601/7122 with 9 missing, elapsed time: 6.418
Imputing row 701/7122 with 11 missing, elapsed time: 6.444
Imputing row 801/7122 with 11 missing, elapsed time: 6.470
Imputing row 901/7122 with 8 missing, elapsed time: 6.490
Imputing row 1001/7122 with 12 missing, elapsed time: 6.516
Imputing row 1101/7122 with 11 missing, elapsed time: 6.548
Imputing row 1201/7122 with 12 missing, elapsed time: 6.568
Imputing row 1301/7122 with 7 missing, elapsed time: 6.602
Imputing row 1401/7122 with 1 missing, elapsed time: 6.620
Imputing row 1501/7122 with 4 missing, elapsed time: 6.643
Imputing row 1601/7122 with 4 missing, elapsed time: 6.666
Imp

In [ ]:
dfe6filled

,iso_code,country,year,trade_co2,cement_co2,coal_co2,flaring_co2,gas_co2,oil_co2,other_industry_co2,consumption_co2,total_ghg,total_ghg_excluding_lucf,methane,nitrous_oxide,population,gdp,primary_energy_consumption
11,AFG,Afghanistan,1960.0,4.825121,0.018,0.127,0.468075,6.331716,0.269,0.019286,52.630406,43.612099,26.255411,18.762165,6.944695,8996967.0,1.303325e+10,2.949846
12,AFG,Afghanistan,1961.0,2.799278,0.022,0.176,0.548903,7.603425,0.293,0.020133,33.225018,43.999025,26.648972,19.038970,7.049714,9169406.0,1.314629e+10,3.017879
13,AFG,Afghanistan,1962.0,5.166143,0.029,0.297,0.694566,9.074737,0.363,0.021839,54.138556,44.214932,26.863575,19.168252,7.099816,9351442.0,1.336763e+10,3.131269
14,AFG,Afghanistan,1963.0,5.396530,0.051,0.264,0.475554,10.127002,0.392,0.027291,59.306020,44.427530,27.077185,19.308069,7.149380,9543200.0,1.363030e+10,3.257176
15,AFG,Afghanistan,1964.0,2.222403,0.062,0.300,0.586721,18.013322,0.476,0.033849,21.192878,44.667891,27.308836,19.459534,7.211517,9744772.0,1.387050e+10,3.278518
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25178,ZWE,Zimbabwe,2008.0,1.575000,0.154,5.958,1.531992,4.544799,1.608,0.039210,9.295000,35.350000,24.150000,10.560000,5.970000,12379553.0,1.505022e+10,41.277000
25179,ZWE,Zimbabwe,2009.0,2.595000,0.270,3.559,1.526867,14.497649,1.648,0.036948,8.071000,36.270000,25.170000,10.920000,6.360000,12526964.0,1.627576e+10,41.112000
25180,ZWE,Zimbabwe,2010.0,1.497000,0.308,5.632,1.465790,12.835969,1.938,0.026987,9.375000,39.300000,28.080000,11.450000,6.770000,12697728.0,1.791838e+10,46.599000
25181,ZWE,Zimbabwe,2011.0,1.046000,0.382,6.101,1.452206,11.580721,3.261,0.041595,10.790000,42.040000,30.550000,11.900000,6.950000,12894323.0,1.955407e+10,51.260000


In [ ]:
dfe6

,iso_code,country,year,trade_co2,cement_co2,coal_co2,flaring_co2,gas_co2,oil_co2,other_industry_co2,consumption_co2,total_ghg,total_ghg_excluding_lucf,methane,nitrous_oxide,population,gdp,primary_energy_consumption
11,AFG,Afghanistan,1960,NaN,0.018,0.127,NaN,NaN,0.269,NaN,NaN,NaN,NaN,NaN,NaN,8996967.0,1.303325e+10,NaN
12,AFG,Afghanistan,1961,NaN,0.022,0.176,NaN,NaN,0.293,NaN,NaN,NaN,NaN,NaN,NaN,9169406.0,1.314629e+10,NaN
13,AFG,Afghanistan,1962,NaN,0.029,0.297,NaN,NaN,0.363,NaN,NaN,NaN,NaN,NaN,NaN,9351442.0,1.336763e+10,NaN
14,AFG,Afghanistan,1963,NaN,0.051,0.264,NaN,NaN,0.392,NaN,NaN,NaN,NaN,NaN,NaN,9543200.0,1.363030e+10,NaN
15,AFG,Afghanistan,1964,NaN,0.062,0.300,NaN,NaN,0.476,NaN,NaN,NaN,NaN,NaN,NaN,9744772.0,1.387050e+10,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25178,ZWE,Zimbabwe,2008,1.575,0.154,5.958,NaN,NaN,1.608,NaN,9.295,35.35,24.15,10.56,5.97,12379553.0,1.505022e+10,41.277
25179,ZWE,Zimbabwe,2009,2.595,0.270,3.559,NaN,NaN,1.648,NaN,8.071,36.27,25.17,10.92,6.36,12526964.0,1.627576e+10,41.112
25180,ZWE,Zimbabwe,2010,1.497,0.308,5.632,NaN,NaN,1.938,NaN,9.375,39.30,28.08,11.45,6.77,12697728.0,1.791838e+10,46.599
25181,ZWE,Zimbabwe,2011,1.046,0.382,6.101,NaN,NaN,3.261,NaN,10.790,42.04,30.55,11.90,6.95,12894323.0,1.955407e+10,51.260


In [ ]:
dfe6filled.to_csv('owid-co2-data-final.csv')

## PREPARING PSD DATA FOR INTEGRATION

In [ ]:
crops

,Commodity_Code,Commodity_Description,Country_Code,Country_Name,Market_Year,Calendar_Year,Month,Attribute_ID,Attribute_Description,Unit_ID,Unit_Description,Value
0,430000,Barley,AF,Afghanistan,1960,2006,7,4,Area Harvested,4,(1000 HA),350.00
1,430000,Barley,AF,Afghanistan,1960,2006,7,20,Beginning Stocks,8,(1000 MT),0.00
2,430000,Barley,AF,Afghanistan,1960,2006,7,125,Domestic Consumption,8,(1000 MT),378.00
3,430000,Barley,AF,Afghanistan,1960,2006,7,176,Ending Stocks,8,(1000 MT),0.00
4,430000,Barley,AF,Afghanistan,1960,2006,7,88,Exports,8,(1000 MT),0.00
...,...,...,...,...,...,...,...,...,...,...,...,...
535315,410000,Wheat,RH,Zimbabwe,2021,2022,2,86,Total Supply,8,(1000 MT),490.00
535316,410000,Wheat,RH,Zimbabwe,2021,2022,2,113,TY Exports,8,(1000 MT),0.00
535317,410000,Wheat,RH,Zimbabwe,2021,2022,2,84,TY Imp. from U.S.,8,(1000 MT),0.00
535318,410000,Wheat,RH,Zimbabwe,2021,2022,2,81,TY Imports,8,(1000 MT),150.00


In [ ]:
crops = crops.drop(['Country_Code', 'Calendar_Year', 'Month', 'Attribute_ID', 'Unit_ID', 'Unit_Description'], axis = 1)

In [ ]:
crops_pivot= crops.pivot(index = ['Commodity_Code', 'Commodity_Description', 'Country_Name', 'Market_Year'], columns = 'Attribute_Description', values = 'Value')

In [ ]:
crops_pivot['Exports'] *= 1000
crops_pivot['Imports'] *= 1000
crops_pivot['Area Harvested'] *= 1000
crops_pivot['Beginning Stocks'] *= 1000
crops_pivot['Ending Stocks'] *= 1000
crops_pivot['FSI Consumption'] *= 1000
crops_pivot['Feed Dom. Consumption'] *= 1000
crops_pivot['Production'] *= 1000
crops_pivot['TY Exports'] *= 1000
crops_pivot['TY Imp. from U.S.'] *= 1000
crops_pivot['TY Imports'] *= 1000
crops_pivot['Milling Rate (.9999)'] *= 1000
crops_pivot['Rough Production'] *= 1000

In [ ]:
crops_pivot = crops_pivot.drop(['Domestic Consumption', 'Total Distribution', 'Total Supply', 'Yield'], axis = 1)

In [ ]:
crops_pivot

Attribute_Description                                            Area Harvested  \
Commodity_Code Commodity_Description Country_Name   Market_Year                   
410000         Wheat                 Afghanistan    1960              2230000.0   
                                                    1961              2230000.0   
                                                    1962              2341000.0   
                                                    1963              2341000.0   
                                                    1964              2345000.0   
...                                                                         ...   
459900         Mixed Grain           United Kingdom 2017                52000.0   
                                                    2018                51000.0   
                                                    2019                51000.0   
                                                    2020                44000.0   
                                                    2021                50000.0   

Attribute_Description                                            Beginning Stocks  \
Commodity_Code Commodity_Description Country_Name   Market_Year                     
410000         Wheat                 Afghanistan    1960                      0.0   
                                                    1961                      0.0   
                                                    1962                      0.0   
                                                    1963                      0.0   
                                                    1964                      0.0   
...                                                                           ...   
459900         Mixed Grain           United Kingdom 2017                      0.0   
                                                    2018                      0.0   
                                                    2019                      0.0   
                                                    2020                      0.0   
                                                    2021                      0.0   

Attribute_Description                                            Ending Stocks  \
Commodity_Code Commodity_Description Country_Name   Market_Year                  
410000         Wheat                 Afghanistan    1960                   0.0   
                                                    1961                   0.0   
                                                    1962                   0.0   
                                                    1963                   0.0   
                                                    1964                   0.0   
...                                                                        ...   
459900         Mixed Grain           United Kingdom 2017                   0.0   
                                                    2018                   0.0   
                                                    2019                   0.0   
                                                    2020                   0.0   
                                                    2021                   0.0   

Attribute_Description                                            Exports  \
Commodity_Code Commodity_Description Country_Name   Market_Year            
410000         Wheat                 Afghanistan    1960             0.0   
                                                    1961             0.0   
                                                    1962             0.0   
                                                    1963             0.0   
                                                    1964             0.0   
...                                                                  ...   
459900         Mixed Grain           United Kingdom 2017             0.0   
                                                    2018       

In [ ]:
crops_pivot.to_csv('PSD_Grains_Final.csv')

In [ ]:
crops_final = pd.read_csv('PSD_Grains_Final.csv')

In [ ]:
crops_final

,Commodity_Code,Commodity_Description,Country_Name,Market_Year,Area Harvested,Beginning Stocks,Ending Stocks,Exports,FSI Consumption,Feed Dom. Consumption,Imports,Milling Rate (.9999),Production,Rough Production,TY Exports,TY Imp. from U.S.,TY Imports
0,410000,Wheat,Afghanistan,1960,2230000.0,0.0,0.0,0.0,2330000.0,0.0,51000.0,NaN,2279000.0,NaN,0.0,46000.0,51000.0
1,410000,Wheat,Afghanistan,1961,2230000.0,0.0,0.0,0.0,2294000.0,0.0,15000.0,NaN,2279000.0,NaN,0.0,15000.0,15000.0
2,410000,Wheat,Afghanistan,1962,2341000.0,0.0,0.0,0.0,2286000.0,0.0,7000.0,NaN,2279000.0,NaN,0.0,7000.0,7000.0
3,410000,Wheat,Afghanistan,1963,2341000.0,0.0,0.0,0.0,2027000.0,0.0,80000.0,NaN,1947000.0,NaN,0.0,56000.0,80000.0
4,410000,Wheat,Afghanistan,1964,2345000.0,0.0,0.0,0.0,2310000.0,0.0,60000.0,NaN,2250000.0,NaN,0.0,52000.0,60000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35683,459900,Mixed Grain,United Kingdom,2017,52000.0,0.0,0.0,0.0,0.0,119000.0,0.0,NaN,119000.0,NaN,0.0,0.0,0.0
35684,459900,Mixed Grain,United Kingdom,2018,51000.0,0.0,0.0,0.0,0.0,169000.0,0.0,NaN,169000.0,NaN,0.0,0.0,0.0
35685,459900,Mixed Grain,United Kingdom,2019,51000.0,0.0,0.0,0.0,0.0,168000.0,0.0,NaN,168000.0,NaN,0.0,0.0,0.0
35686,459900,Mixed Grain,United Kingdom,2020,44000.0,0.0,0.0,0.0,0.0,149000.0,0.0,NaN,149000.0,NaN,0.0,0.0,0.0


In [ ]:
yearIndex = crops_final[crops_final['Market_Year'] < 1960].index
yearIndex2 = crops_final[crops_final['Market_Year'] > 2012].index

In [ ]:
crops_final = crops_final.drop(yearIndex)
crops_final = crops_final.drop(yearIndex2)

In [ ]:
crops_final = crops_final[crops_final.Country_Name.isin(values) == True]

In [ ]:
crops_final.describe()

,Commodity_Code,Market_Year,Area Harvested,Beginning Stocks,Ending Stocks,Exports,FSI Consumption,Feed Dom. Consumption,Imports,Milling Rate (.9999),Production,Rough Production,TY Exports,TY Imp. from U.S.,TY Imports
count,26408.000000,26408.000000,2.640800e+04,2.640800e+04,2.640800e+04,2.640800e+04,2.128500e+04,2.128500e+04,2.640800e+04,5.123000e+03,2.640800e+04,5.123000e+03,2.640800e+04,2.640800e+04,2.640800e+04
mean,434230.976598,1987.367124,1.123800e+06,6.477055e+05,6.570511e+05,3.376322e+05,1.315333e+06,1.014643e+06,2.915801e+05,5.166907e+06,2.513342e+06,4.476207e+06,3.366595e+05,1.118279e+05,2.924962e+05
std,17613.050032,15.032558,3.852741e+06,4.878211e+06,4.919718e+06,2.601416e+06,6.114271e+06,6.958607e+06,1.017492e+06,2.701517e+06,1.313920e+07,1.961099e+07,2.596485e+06,6.567688e+05,1.020922e+06
min,410000.000000,1960.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,422110.000000,1975.000000,9.000000e+03,0.000000e+00,0.000000e+00,0.000000e+00,2.600000e+04,0.000000e+00,0.000000e+00,6.000000e+06,1.100000e+04,5.000000e+03,0.000000e+00,0.000000e+00,0.000000e+00
50%,430000.000000,1988.000000,8.500000e+04,0.000000e+00,0.000000e+00,0.000000e+00,1.320000e+05,2.200000e+04,2.100000e+04,6.500000e+06,1.180000e+05,1.140000e+05,0.000000e+00,0.000000e+00,2.100000e+04
75%,452000.000000,2000.000000,5.000000e+05,9.300000e+04,9.900000e+04,2.000000e+03,6.070000e+05,2.500000e+05,1.570000e+05,6.660000e+06,7.220000e+05,7.810000e+05,2.000000e+03,1.100000e+04,1.572500e+05
max,459900.000000,2012.000000,4.553700e+07,1.240010e+08,1.240010e+08,6.191300e+07,1.633820e+08,1.557510e+08,1.686300e+07,7.399000e+06,3.319210e+08,2.065310e+08,6.180100e+07,1.636100e+07,1.686300e+07


In [ ]:
(crops_final == 0).sum()

Commodity_Code               0
Commodity_Description        0
Country_Name                 0
Market_Year                  0
Area Harvested            4065
Beginning Stocks         13604
Ending Stocks            13440
Exports                  19054
FSI Consumption           1391
Feed Dom. Consumption     8778
Imports                   8780
Milling Rate (.9999)      1093
Production                4071
Rough Production          1117
TY Exports               19089
TY Imp. from U.S.        17180
TY Imports                8744
dtype: int64

In [ ]:
crops_final.isnull().sum()
# Missing Values Can Be Imputed with 0, because missing values are not at random, and are based on the commodity
# FSI Consumption and Feed Dom. Consumption are missing 5176 values, because those values are correlated to the number of rows pertaining to Rice columns
# Milling Rate and Rough Production are missing 21285 values, because those values are correlated to the number of rows pertaining to all commodities, but rice
# These values are only missing as a result of the pivot that was done to the dataframe.

Commodity_Code               0
Commodity_Description        0
Country_Name                 0
Market_Year                  0
Area Harvested               0
Beginning Stocks             0
Ending Stocks                0
Exports                      0
FSI Consumption           5123
Feed Dom. Consumption     5123
Imports                      0
Milling Rate (.9999)     21285
Production                   0
Rough Production         21285
TY Exports                   0
TY Imp. from U.S.            0
TY Imports                   0
dtype: int64

In [ ]:
21285 + 5176

26461

In [ ]:
crops_final[crops_final['Commodity_Description'] != 'Rice, Milled'].count()

Commodity_Code           21285
Commodity_Description    21285
Country_Name             21285
Market_Year              21285
Area Harvested           21285
Beginning Stocks         21285
Ending Stocks            21285
Exports                  21285
FSI Consumption          21285
Feed Dom. Consumption    21285
Imports                  21285
Milling Rate (.9999)         0
Production               21285
Rough Production             0
TY Exports               21285
TY Imp. from U.S.        21285
TY Imports               21285
dtype: int64

In [ ]:
crops_final = crops_final.fillna(0)

In [ ]:
crops_final.to_csv('PSD_Grains_Final.csv')

## LATITUDE AND LONGITUDE DATA PREPARATION FOR INTEGRATIONcoor = pd.read_csv('world_country_and_usa_states_latitude_and_longitude_values.csv')

In [ ]:
coor = pd.read_csv('world_country_and_usa_states_latitude_and_longitude_values.csv')

In [ ]:
coor = coor.drop(['usa_state_code', 'usa_state_latitude', 'usa_state_longitude', 'usa_state', 'country_code'], axis=1)

In [ ]:
coor

,latitude,longitude,country
0,42.546245,1.601554,Andorra
1,23.424076,53.847818,United Arab Emirates
2,33.939110,67.709953,Afghanistan
3,17.060816,-61.796428,Antigua and Barbuda
4,18.220554,-63.068615,Anguilla
...,...,...,...
240,15.552727,48.516388,Yemen
241,-12.827500,45.166244,Mayotte
242,-30.559482,22.937506,South Africa
243,-13.133897,27.849332,Zambia


In [ ]:
coor_up = coor[coor.country.isin(values) == True]

In [ ]:
coor_up

,latitude,longitude,country
1,23.424076,53.847818,United Arab Emirates
2,33.939110,67.709953,Afghanistan
5,41.153332,20.168331,Albania
6,40.069099,45.038189,Armenia
8,-11.202692,17.873887,Angola
...,...,...,...
235,14.058324,108.277199,Vietnam
240,15.552727,48.516388,Yemen
242,-30.559482,22.937506,South Africa
243,-13.133897,27.849332,Zambia


In [ ]:
coor_up.to_csv('CountryCoord.csv')